# Starter Transition-Mapping Template Notebook


In [103]:
import plotly.express as px
import pandas as pd

import os

### Import a chosen groupMetadataID below.

If we already have the data, load it.


In [104]:
gmID = '3a2a78cc-db21-11ee-a158-97f8443fd730'

# set to False if data is stored as .csv file.
# otherwise, set to True

if os.path.isfile(f"./data/chassis/{gmID}.csv"):
    queryChassis = False
    print("chassis csv file found!")
else:
    queryChassis = True
    print("no chassis csv. Query for data necessary.")

if os.path.isfile(f"./data/best_pose/{gmID}.csv"):
    queryPose = False
    print("best_pose csv file found!")
else:
    queryPose = True
    print("no best_ose csv. Query for data necessary.")

if os.path.isfile(f"./data/metadata/{gmID}.csv"):
    queryMeta = False
    print("metadata csv file found!")
else:
    queryMeta = True
    print("no metadata csv. Query for data necessary.")

chassis csv file found!
best_pose csv file found!
metadata csv file found!


### Connect to database and query


In [105]:
from boto3.dynamodb.conditions import Key, Attr
import boto3

if queryPose or queryChassis or queryMeta:
    # Get the service resource
    dynamodb = boto3.resource('dynamodb')
    # Instantiate a table resource object
    table = dynamodb.Table('ads_passenger_processed')
    table_meta = dynamodb.Table('ads_passenger_processed_metadata')

In [106]:
if not os.path.exists(f"./data/best_pose"):
        os.makedirs(f"./data/best_pose")
if queryPose:

    # query arguments for best_pose data
    keywords = dict(
        IndexName='topic-index',
        KeyConditionExpression=Key('topic').eq(
            '/apollo/sensor/gnss/best_pose'),
        ProjectionExpression="groupMetadataID, #t, solStatus, solType, latitudeStdDev, longitudeStdDev, numSatsTracked, numSatsMulti, numSatsInSolution, differentialAge, solutionAge, latitude, longitude",
        Limit=1500,
        ExpressionAttributeNames={'#t': 'time'},
        FilterExpression=Key('groupMetadataID').eq(f'{gmID}')

    )

    # run initial query and coerce results into a dataframe
    res = table.query(**keywords)
    df_pose = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns')

    done = False
    leek = res["LastEvaluatedKey"]

    while not done:
        try:
            leek = res["LastEvaluatedKey"]
            keywords["ExclusiveStartKey"] = leek
            res = table.query(**keywords)
            tmpDF = pd.DataFrame.from_dict(
                pd.json_normalize(res['Items']), orient='columns')
            df_pose = pd.concat([df_pose, tmpDF])
        except KeyError:
            print("Done querying best_pose data")
            done = True

    df_pose.to_csv(f"./data/best_pose/{gmID}.csv")
    print(f"Saved best_pose data at ./data/best_pose/{gmID}.csv")


else:
    df_pose = pd.read_csv(f"./data/best_pose/{gmID}.csv")
    print("Loaded best_pose data")

df_pose.shape

Loaded best_pose data


(2001, 13)

In [107]:
if not os.path.exists(f"./data/chassis"):
        os.makedirs(f"./data/chassis")

if queryChassis:
    keywords = dict(
        IndexName='topic-index',
        KeyConditionExpression=Key('topic').eq(
            '/apollo/canbus/chassis'),
        Limit=2000,
        FilterExpression=Key('groupMetadataID').eq(gmID)
    )

    res = table.query(**keywords)
    df_chassis = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns')

    done = False
    leek = res["LastEvaluatedKey"]

    while not done:
        try:
            leek = res["LastEvaluatedKey"]
            print(leek)
            keywords["ExclusiveStartKey"] = leek
            res = table.query(**keywords)
            tmpDF = pd.DataFrame.from_dict(
                pd.json_normalize(res['Items']), orient='columns')
            df_chassis = pd.concat([df_chassis, tmpDF])

        except KeyError:
            print(f"done querying for chassis data")
            done = True

    df_chassis.to_csv(f"./data/chassis/{gmID}.csv")
    print(f"Saved chassis data at ./data/chassis/{gmID}.csv")
else:
    df_chassis = pd.read_csv(f"./data/chassis/{gmID}.csv")
    print("Loaded chassis data")

print(df_chassis.shape)

Loaded chassis data
(31903, 31)


In [108]:
if not os.path.exists(f"./data/metadata"):
        os.makedirs(f"./data/metadata")

if queryMeta:

    res = table_meta.query(
        IndexName='groupMetadataID-index',
        KeyConditionExpression=Key("groupMetadataID").eq(gmID),
        ProjectionExpression="groupMetadataID, #o.Weather, #o.#m, #o.Notes",
        Limit=1500,
        ExpressionAttributeNames={"#o": "other", "#m": "Map"},
    )

    df_meta = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns').drop_duplicates()

    df_meta.to_csv(f"./data/metadata/{gmID}.csv")
    print(f"Saved metadata data at ./data/metadata/{gmID}.csv")

else:
    df_meta = pd.read_csv(f"./data/metadata/{gmID}.csv")
    print("Loaded metadata")


print(df_meta.shape)

Loaded metadata
(1, 8)


The output of this box will be messed up. It automatically sets row 0 to True. Not sure why, but it doesn't count as a transition in the end.
We are also counting emergency mode as MANUAL


In [118]:
# read the timeSorted csv
pts = df_chassis.copy()
pts = pts.sort_values('time')
# reduce df to only time and drivingMode
pts = pts[["time", "drivingMode"]]
# function to tell us what state transition it is


def getState(old, new):
    if old == "COMPLETE_AUTO_DRIVE" and new == "COMPLETE_MANUAL":
        return "A->M"
    elif old == "COMPLETE_AUTO_DRIVE" and new == "EMERGENCY_MODE":
        return "A->M"
    elif old == "COMPLETE_MANUAL" and new == "COMPLETE_AUTO_DRIVE":
        return "M->A"
    elif old == "COMPLETE_MANUAL" and new == "EMERGENCY_MODE":
        return "NONE"
    elif old == "EMERGENCY_MODE" and new == "COMPLETE_AUTO_DRIVE":
        return "M->A"
    elif old == "EMERGENCY_MODE" and new == "COMPLETE_MANUAL":
        return "NONE"
    elif old == new:
        return "NONE"
    else:
        return "ERROR"


# we shift the drivingMode column down by 1
pts["shift"] = pts["drivingMode"].shift()
# create transition field
pts['transition'] = (pts["drivingMode"] != pts["shift"])
# set Transition type to NONE for all
pts['transitionType'] = "NONE"
# Set first transition to False, since it was auto True
pts.loc[0, 'transition'] = False
# print(pts['transition'])
# Show counts for transitions
print(pts['transition'].value_counts())
# iterate over all rows and find the ones where transition is true. Then we compare shift and dM to see what
# transition Type we have.
for index, row in pts.iterrows():
    if pts['transition'][index] == True:
        old = pts['shift'][index]
        new = pts['drivingMode'][index]
        pts.loc[index, "transitionType"] = getState(old, new)
print(pts['transitionType'].value_counts())
pts = pts[['time', 'drivingMode', 'transition', 'transitionType']]
pts = pts.sort_values('time')
pts['time'] = pts['time'].astype(float)
pts['time'] = pts['time'].astype('datetime64[ns]')
pts['time'] = pts['time'].astype('datetime64[s]')
pts[['time']]

# show(pts)

transition
False    31854
True        49
Name: count, dtype: int64


transitionType
NONE     31870
M->A        16
A->M        16
ERROR        1
Name: count, dtype: int64


,time
3092,2023-08-17 15:14:25
31671,2023-08-17 15:14:25
14369,2023-08-17 15:14:25
2187,2023-08-17 15:14:25
19724,2023-08-17 15:14:25
...,...
2064,2023-08-17 15:47:47
30421,2023-08-17 15:47:47
1938,2023-08-17 15:47:47
12877,2023-08-17 15:47:47


In [119]:
ll = df_pose.copy()
ll = ll.sort_values('time')
ll['time'] = ll['time'].astype(float)
ll['time'] = ll['time'].astype('datetime64[ns]')
ll['time'] = ll['time'].astype('datetime64[s]')
ll = ll[["groupMetadataID", 'time', 'latitude', 'longitude']]
merged_df = pd.merge(pts, ll, on='time')

In [120]:
merged_df = merged_df.drop_duplicates()
merged_df

,time,drivingMode,transition,transitionType,groupMetadataID,latitude,longitude
0,2023-08-17 15:14:27,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.329602,-82.127926
30,2023-08-17 15:14:28,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.329603,-82.127926
60,2023-08-17 15:14:29,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.329602,-82.127926
91,2023-08-17 15:14:30,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.329603,-82.127926
121,2023-08-17 15:14:31,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.329602,-82.127926
...,...,...,...,...,...,...,...
43707,2023-08-17 15:47:43,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.330517,-82.128211
43737,2023-08-17 15:47:44,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.330517,-82.128211
43767,2023-08-17 15:47:45,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.330517,-82.128211
43786,2023-08-17 15:47:46,EMERGENCY_MODE,False,NONE,3a2a78cc-db21-11ee-a158-97f8443fd730,39.330517,-82.128211


In [121]:
merged_notes_df = pd.merge(merged_df, df_meta, on="groupMetadataID")

### Mapping


In [124]:
# Define your map with your dataframe name first and keep everything else the same
fig = px.scatter_mapbox(merged_df, lat="latitude",
                        lon="longitude",
                        hover_data=["time", "drivingMode", "transitionType"],
                        color="drivingMode",
                        zoom=13,
                        height=1000,
                        width=1200,
                         #color_discrete_map={
                         #   "M->A": "#63ACBE",
                         #   "A->M": "#601A4A",
                         #   "ERROR": "#EE442F",
                         #   "NONE":"yellowgreen",
                         #}
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }

                        )

# Layout settings
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.update_traces(marker=dict(size=8))

# Display the map
fig.show()

/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

